In [27]:
import torch
from torch_geometric.loader import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
import argparse
import time
import numpy as np
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from train import train,eval
from model import GNN

In [28]:
def main(dataset_name,batch_size,num_layer,emb_dim,drop_ratio):
    epochs = 150
    batch_size = int(batch_size)
    num_layer = int(num_layer)
    emb_dim = int(emb_dim)
    epochs = int(epochs)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    dataset = PygGraphPropPredDataset(name = dataset_name)
    split_idx = dataset.get_idx_split()
    evaluator = Evaluator(dataset_name)
    train_loader = DataLoader(dataset[split_idx["train"]], batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(dataset[split_idx["valid"]], batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(dataset[split_idx["test"]], batch_size=batch_size, shuffle=False)

    model = GNN(num_tasks = dataset.num_tasks, num_layer = num_layer, emb_dim = emb_dim, drop_ratio = drop_ratio).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    valid_acc_list = []
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, dataset.task_type)
        train_perf = eval(model, device, train_loader, evaluator)
        valid_perf = eval(model, device, valid_loader, evaluator)
        test_perf = eval(model, device, test_loader, evaluator)
        valid_acc_list.append(valid_perf['rocauc'])
    return max(valid_acc_list)

In [29]:
from functools import partial

database = 'ogbg-molhiv'
fit_with_partial = partial(main,database)

In [30]:
pbounds = {'batch_size':(10,100),'num_layer':(2,7),'emb_dim':(100,200),'drop_ratio':(0.2,0.5)}

In [31]:
from bayes_opt import BayesianOptimization

optimizer = BayesianOptimization(
    f=fit_with_partial,
    pbounds=pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

In [ ]:
optimizer.maximize(init_points=10, n_iter=10,)
for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

print(optimizer.max)
f = open("params.txt", "a")
f.write(optimizer.max)
f.close()

|   iter    |  target   | batch_... | drop_r... |  emb_dim  | num_layer |
-------------------------------------------------------------------------


Iteration:  40%|████      | 283/701 [00:01<00:02, 199.04it/s]